<a href="https://colab.research.google.com/github/Narusi/My-Projects/blob/master/ss%20dzivokli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [0]:
def FlatDetails(link):
    import requests
    from bs4 import BeautifulSoup

    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    date = ''    
    for x in soup.find_all('td',"msg_footer",):
        if x.text.find('Date') >= 0:
            date = (x.text[6:])    
    return date

def GetProperties(path, district='Unknown', pages=10, result='prices'):
    import csv
    
    #GET ALL PROPERTIES DISTRICT BY DISTRICT
    properties = []
    deals = ['hand_over/', 'sell/']
    postID = []
    links = {}
    
    invalid_items = []
    
    try:
        for dd in deals:
            if dd == 'sell/':
                tradeType = 'SELL'
            else:
                tradeType = 'RENT'

            for n in range(1, pages):
                
                if n == 1:
                    fullPath = path + dd
                else:
                    fullPath = path + dd + 'page' + str(n) + '.html'
                
                #print('Reading: ' + fullPath)
                r = requests.get(fullPath)
                soup = BeautifulSoup(r.text, 'html.parser')

                raw = soup.find_all('tr')
                #print('Found {} entries in this page'.format(len(raw)))
                for rr in raw:
                    temp = []

                    for ss in rr.find_all('td'):
                        temp.append(ss.text)
                    
                    #print(len(temp))
                
                    if len(temp) == 10 and \
                    str(rr.get('id') + temp[3] + temp[4] + temp[5] + \
                        temp[6] + temp[7]) not in postID:
                        
                        if str(temp[8]) == '-':
                            sqmPrice = 0.00
                        else:
                            sqmPrice = float(str(temp[8]).replace(' €','').replace(',',''))
                        
                        if len(temp[9])<2 and temp[9]=='-':
                            price = 0.00
                        elif str(temp[9]).find('/')<0 and temp[9]!='buy ':
                            price = float(str(temp[9]).replace(' €','').replace(',','')\
                                          .encode("ascii", "ignore"))
                        else:
                            if str(temp[9]).find('/day')<0 and\
                            str(temp[9]).find('/week')<0\
                            and temp[9]!='buy ':
                                price = float(str(temp[9]).replace('/mon.', '')\
                                              .replace(' €','').replace(',','')\
                                             .encode("ascii", "ignore"))
                            else:
                                price = 0.00
                        
                        if tradeType == 'RENT':    
                            altPrice = float(price) * 120
                            altSQMprice = float(sqmPrice) * 120
                        else:
                            altPrice = price
                            altSQMprice = sqmPrice
                                                
                        for x in rr.find_all('a'):
                            links[rr.get('id')] = 'https://www.ss.com/' +\
                            str(x.get('href'))
                        
                        #print(district, streetName, altPrice, altSQMprice)
                        if len(str(temp[4])) > 2 or len(str(temp[4])) == 0:
                            rooms = 0
                        elif len(str(temp[4])) > 0:
                            rooms = int(temp[4])
                        
                        size = temp[5]
                        floor = str(temp[6]).split('/')[0]
                        maxFloor = str(temp[6]).split('/')[-1]
                        project = str(temp[7])

                        if len(temp[3].split(' ')[0])>3:
                            streetName =  temp[3].split(' ')[0]
                            streetNo = temp[3].split(' ')[-1]
                        elif len(temp[3].split(' ')) > 2:
                            streetName =  str(temp[3].split(' ')[0]) + \
                            str(temp[3].split(' ')[1])
                            streetNo = temp[3].split(' ')[-1]
                        else:
                            streetName =  str(temp[3])
                            streetNo = ''                              
                        
                        postDate = FlatDetails(links[rr.get('id')])
                        comment = str(temp[2])
                        pID = str(str(rr.get('id'))[4:] + temp[3] + temp[4] + \
                                  temp[5] + temp[6] + temp[7])
                        
                        properties.append([pID, tradeType, comment, links[rr.get('id')], \
                                           district, streetName, streetNo, rooms, size, \
                                           floor, maxFloor, project, postDate, \
                                           sqmPrice, altSQMprice, price, altPrice])
                        
                        postID.append(str(rr.get('id') + temp[3] + temp[4] + \
                                          temp[5] + temp[6] + temp[7]))
                    else:
                        invalid_items.append(temp)
    finally:
        with open('JUNK.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=';', quoting=csv.QUOTE_MINIMAL)
            for item in invalid_items:
                #Special characters would normaly crash 
                #any string manipulation so we need to encode them:
                csvwriter.writerow(str(item).encode("ascii", "ignore"))
        
        postID = []
        
    return properties

In [0]:
districts = {}

#GET ALL DISTRICTS AND THEIR LINKS
r = requests.get('https://www.ss.com/en/real-estate/flats/riga/')
soup = BeautifulSoup(r.text, 'html.parser')

raw = soup.find_all('a','a_category')
for r in raw:
    districts[r.text] = 'https://www.ss.com' + r.get('href')
    #print(r.text)

In [0]:
rajoni = ['Centre','Grizinkalns','Sarkandaugava','VEF','Teika']

cols = ['ID', 'Deal Type', 'Comment', 'Link', 'District', 'Street Name',
        'Street No.', 'Rooms', 'Size','Floor', 'Max. Floor','Project',
        'Post Date', 'Price of sqm', 'Alt. Price of sqm','Total Price', 'Alt. Price']

numCols = ['Rooms', 'Size','Floor', 'Max. Floor','Price of sqm',
           'Alt. Price of sqm','Total Price', 'Alt. Price']

In [0]:
temp = []

for r in rajoni:
    print('Processing ', r)
    tData = GetProperties(districts[r], district = r)
    for tt in tData:
        count = 0
        for t in tt:
            if t != '':
                count += 1
                
        if count == len(tt):
            temp.append(tt)

ipasumi = pd.DataFrame(temp, columns=cols).dropna()
for col in numCols:
    try:
        ipasumi[col] = pd.to_numeric(ipasumi[col])
    except:
        print(col)
        print(ipasumi[col].to_list())

ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])

for raj in rajoni:
    print(raj)
    print(ipasumi[ipasumi['District'] == raj].describe())

Processing  Centre


In [0]:
for key in districts:    
    if key != 'All announcements':
        tData = GetProperties(districts[key], district = key, pages=100)
        for ii in tData:
            temp.append(ii)
        print(key)

ipasumi = pd.DataFrame(temp, columns=cols).dropna()

In [0]:
ipasumi = ipasumi[np.logical_and(ipasumi['Total Price'] > 0,
                                 ipasumi['Rooms'] != 'Other')]
for col in numCols:
    ipasumi[col] = pd.to_numeric(ipasumi[col])

ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])
ipasumi = ipasumi.dropna()
ipasumi.info()

In [0]:
#Dropping comment as csv cannot properly store long comment fields:
ipasumi.drop(ipasumi['Comment'], axis=1).to_csv('ssDzivokli.csv', index=False,
                                                sep = ';', decimal = ',')

print('Mean price for square meter in New project flat: {} eiro'.format(ipasumi['Alt. Price of sqm'].where(ipasumi['Project']=='New').mean()))
ipasumi['Project'].count()

In [0]:
try:
    conn = sqlite3.connect('miniSS.db')

    try:
        data = pd.read_sql('SELECT * FROM PropertiesRAW', \
                           conn, index_col='ID').drop('index', axis = 1)

        ipasumi = ipasumi.append(data).sort_values(by=['Post Date'], ascending=False)
        ipasumi = ipasumi.drop_duplicates(['District', 'Street Name', \
                                           'Street No.', 'Rooms', 'Size',\
                                           'Floor', 'Max. Floor','Project'])

        conn.cursor().execute('DROP TABLE PropertiesRAW')
        conn.commit()
        ipasumi.to_sql('PropertiesRAW', conn)
    except:
        ipasumi.to_sql('PropertiesRAW', conn)

    conn.close
    data = pd.DataFrame(data.values, columns=cols)
    data['Post Date'] = pd.to_datetime(data['Post Date'])
    data.to_csv('data.csv', index=False, sep = ';', decimal = ',')
except:
    ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])
    ipasumi.to_csv('data.csv', index=False, sep = ';', decimal = ',')

In [0]:
pd.DataFrame(GetProperties(districts['Imanta'], district = 'Imanta', pages=100),
             columns=cols)['Rooms']